In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd

import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm

from shapely.geometry import Point
from shapely.geometry import mapping

In [ ]:
lon_min, lon_max = 5.875, 20.125
lat_min, lat_max = 35.875, 48.125

In [ ]:
# Resample the precipitation data to monthly averages

prec = xr.open_dataset('eobs/eobs_prec.nc').sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))
monthly_prec = prec.resample(time='ME').mean()

monthly_prec_df = monthly_prec.to_dataframe().reset_index()
monthly_prec_df['geometry'] = [Point(lon, lat) for lon, lat in zip(monthly_prec_df['longitude'], monthly_prec_df['latitude'])]
monthly_prec_gdf = gpd.GeoDataFrame(monthly_prec_df, geometry='geometry')

monthly_prec_gdf = monthly_prec_gdf.set_crs(epsg=4326, inplace=True)

In [ ]:
gdf = gpd.read_file('tweets/NUTS_RG_60M_2021_4326.shp', encoding='latin1')
gdf_italy = gdf[gdf['CNTR_CODE'] == 'IT']
nuts3_gdf = gdf_italy[gdf_italy['LEVL_CODE'] == 3]

In [ ]:
# Check NUTS3 shapefiles

nuts3_gdf.plot()

In [ ]:
# Group the joined data by NUTS3 region and time to calculate the average monthly precipitation on the NUTS3 level

monthly_prec_nuts3 = gpd.sjoin(monthly_prec_gdf, nuts3_gdf, how='left', predicate='within')

agg_prec = monthly_prec_nuts3.groupby(['NUTS_ID', 'time'])['rr'].mean().unstack()
agg_prec = agg_prec.fillna(0)

In [ ]:
agg_prec.to_csv('eobs/nuts3_prec_1950_2023.csv')

In [ ]:
# index computation conducted in R
# header added manually after computation

In [ ]:
spi_data = pd.read_csv('eobs/nuts_spi3.csv')

In [ ]:
# Plot NUTS3 SPI across months

def spi_plot_all(months):

    fig, axs = plt.subplots(3, 4, figsize=(20, 15))
    norm = TwoSlopeNorm(vmin=-2.5, vcenter=0, vmax=2.5)
    
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    for i, month_index in enumerate(months):
        row = i // 4  # Determine the row of the subplot
        col = i % 4   # Determine the column of the subplot
        
        spi_target_month = spi_data.columns[month_index]
        merged_gdf = gdf_italy.merge(spi_data[['NUTS_ID', spi_target_month]], on='NUTS_ID', how='left')

        merged_gdf.plot(column=spi_target_month, cmap='RdYlGn', norm=norm, linewidth=0.8, 
                        ax=axs[row, col], edgecolor='black', legend=False)

        axs[row, col].set_title(f"{month_names[i]}")
        axs[row, col].axis('off')

    # Add a common colorbar for all subplots
    cbar_ax = fig.add_axes([0.92, 0.3, 0.02, 0.4])  # Position of colorbar
    sm = plt.cm.ScalarMappable(cmap='RdYlGn', norm=norm)
    sm._A = []
    fig.colorbar(sm, cax=cbar_ax)

    plt.tight_layout(rect=[0, 0, 0.9, 0.95])  # Adjust layout to fit the colorbar and title
    plt.show()

In [ ]:
months = list(range(865, 877))
spi_plot_all(months)